In [5]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score

In [6]:
# Load Dataset
df = pd.read_csv("delhi_routes.csv")

# Handling Missing Values
df.fillna("Unknown", inplace=True)

# Encoding Categorical Features
label_encoders = {}
for col in ["Start", "Destination", "Via Road", "Route Type"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Features and Targets
X = df[["Start", "Destination"]].values
y = df.drop(columns=["Start", "Destination", "Congestion Points", "Waterlogging Points"]).values

# Normalize Target Variables
scaler = StandardScaler()
y = scaler.fit_transform(y)

# Save Encoders and Scaler
joblib.dump(label_encoders, "label_encoders.pkl")
joblib.dump(scaler, "scaler.pkl")

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build Optimized ANN Model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(y.shape[1], activation='linear')
])

model.compile(optimizer=Adam(learning_rate=0.005), loss='mean_squared_error', metrics=['mae'])

# Train Model with Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, epochs=300, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stop], verbose=1)

# Evaluate Model
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

# Save Model
model.save("ann_route_prediction.h5")

# Load Model and Run Example Test
model = load_model("ann_route_prediction.h5")
label_encoders = joblib.load("label_encoders.pkl")
scaler = joblib.load("scaler.pkl")

example_start = "Connaught Place, Delhi"
example_dest = "Noida, Delhi"
example_input = np.array([[label_encoders["Start"].transform([example_start])[0], 
                            label_encoders["Destination"].transform([example_dest])[0]]])
example_pred = model.predict(example_input)
example_output = scaler.inverse_transform(example_pred)[0]

predicted_details = {
    "Start": example_start,
    "Destination": example_dest,
    "Via Road": label_encoders["Via Road"].inverse_transform([int(example_output[0])])[0],
    "Distance (km)": example_output[1],
    "Signals": int(example_output[2]),
    "Congestion Count": int(example_output[3]),
    "Waterlogging Count": int(example_output[4]),
    "Route Type": label_encoders["Route Type"].inverse_transform([int(example_output[5])])[0],
    "Time (Car)": example_output[6],
    "Time (Bike)": example_output[7],
    "Time (Cycle)": example_output[8],
    "Time (Bus)": example_output[9],
    "Time (Metro)": example_output[10],
    "Total Time": example_output[11],
    "Min Time": example_output[12]
}

print("Predicted Route Details:")
for key, value in predicted_details.items():
    print(f"{key}: {value}")


Epoch 1/300


C:\Users\adars\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.0411 - mae: 0.8722 - val_loss: 0.9982 - val_mae: 0.8537
Epoch 2/300
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9872 - mae: 0.8485 - val_loss: 0.9974 - val_mae: 0.8538
Epoch 3/300
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9904 - mae: 0.8504 - val_loss: 0.9972 - val_mae: 0.8544
Epoch 4/300
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9946 - mae: 0.8531 - val_loss: 0.9971 - val_mae: 0.8542
Epoch 5/300
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9866 - mae: 0.8477 - val_loss: 0.9975 - val_mae: 0.8549
Epoch 6/300
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9923 - mae: 0.8494 - val_loss: 0.9970 - val_mae: 0.8545
Epoch 7/300
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0064 - mae: 0.8574 - val_loss: 0.9977 - val_mae: 0.8537
Epoch 8/300
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9974 - mae: 0.8536 - val_loss: 0.9977 - val_mae: 0.8540
Epoch 9/300
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss:

R² Score: -0.0002939636232248517


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted Route Details:
Start: Connaught Place, Delhi
Destination: Noida, Delhi
Via Road: NH44
Distance (km): 27.28095054626465
Signals: 8
Congestion Count: 1
Waterlogging Count: 0
Route Type: Internal Road
Time (Car): 70.90190887451172
Time (Bike): 55.53883743286133
Time (Cycle): 129.72499084472656
Time (Bus): 64.28028869628906
Time (Metro): 42.468170166015625
Total Time: 11.168441772460938
Min Time: 4.6823225021362305
